# 

In [5]:
import openai
from openai import OpenAI

# Point to the server
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

In [45]:
# Sentences to classify
sentences = [
    "I hate you and I hope you fail.",
    "What a beautiful day to go for a walk!",
    "Your idea is stupid and nobody cares."
]

sentences = [
    "#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon @USER URL",
    "#ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.",
]

# Build a single prompt
prompt = "Classify each of the following sentences as 'hate' or 'non-hate':\n\n"
for i, s in enumerate(sentences, 1):
    prompt += f"{i}. {s}\n"

prompt += "\nReturn the results in the format:\n<number>. <label>\n"

# Make one request for all sentences
response = client.completions.create(
    model="local model",  # currently unused
    prompt=prompt,
    max_tokens=50,
    temperature=0,
    stop=["Classify", "\n\n"]
)

# Print the raw model output
print(response.choices[0].text.strip())

1. hate
2. non-hate


In [66]:
def evaluate(prompt, data):
    
    sentences = []
    for id, row in data.iterrows():
        sentences.append(row['text'])

    # Build a single prompt
    prompt = "Classify each of the following sentences as 'hate' or 'non-hate':\n\n"
    for i, s in enumerate(sentences, 1):
        prompt += f"{i}. {s}\n"

    prompt += "\nReturn the results in the format:\n<number>. <label>\n"

    # Make one request for all sentences
    response = client.completions.create(
        model="local model",  # currently unused
        prompt=prompt,
        max_tokens=50,
        temperature=0,
        stop=["Classify", "\n\n"]
    )

    # Print the raw model output
    print(response.choices[0].text.strip())

In [73]:
import pandas as pd

# evaluation data
data = pd.read_csv("data/olid-test.csv")
data.head()

,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1


In [75]:
data.head(10)

,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1
5,83681,. . . What the fuck did he do this time?,1
6,96874,#RAP is a form of ART! Used to express yoursel...,0
7,65507,@USER Do you get the feeling he is kissing @US...,1
8,78910,5 Tips to Enhance Audience Connection on Faceb...,0
9,46363,#BiggBossTamil janani won the task. She is goi...,0


In [78]:
# Prompts

prompts = {
    "prompt1": "Classify each of the following sentences as 'hate' or 'non-hate':\n\n",
    "prompt2": "",
    "prompt3": "",
}

evaluate(prompts["prompt1"], data.head(10))

1. hate
2. non-hate
3. hate
4. non-hate
5. hate
6. hate
7. non-hate
8. hate
9. non-hate
10. non-hate
